In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ADD THE LIBRARIES YOU'LL NEED
import gensim
import gensim.downloader as gensim_api
import pandas as pd
import numpy as np
from sklearn import metrics
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import models, layers, preprocessing
import re
import nltk
nltk.download('punkt', quiet=True)
# from nltk.tokenize import word_tokenize
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords
pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
# tokenizer = Tokenizer()
from tensorflow.keras.layers import Embedding
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
def embedding(train_reviews):
  #nlp = gensim_api.load("word2vec-google-news-300")
  corpus = train_reviews
  lst_corpus = []
  for string in corpus:
    lst_words = string.split()
    lst_grams = [" ".join(lst_words[i:i+1]) for i in range(0, len(lst_words), 1)]
    lst_corpus.append(lst_grams)

  nlp = gensim.models.word2vec.Word2Vec(lst_corpus, size=300,   
            window=8, min_count=1, sg=1, iter=30)
  
  ## tokenize text
  tokenizer = Tokenizer(lower=True, split=' ', 
                      oov_token="NaN", 
                      filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
  tokenizer.fit_on_texts(lst_corpus)
  dic_vocabulary = tokenizer.word_index
  vocab_size = len(dic_vocabulary)
  ## create sequence
  lst_text2seq= tokenizer.texts_to_sequences(lst_corpus)
  ## padding sequence
  X_train = pad_sequences(lst_text2seq, 
                      maxlen=29, padding="post", truncating="post")
  
  ## start the matrix (length of vocabulary x vector size) with all 0s
  embeddings = np.zeros((len(dic_vocabulary)+1, 300))
  for word,idx in dic_vocabulary.items():
    ## update the row with vector
    try:
      embeddings[idx] =  nlp[word]
    ## if word not in model then skip and the row stays all 0s
    except:
      pass

  return X_train, embeddings, vocab_size

def token(train_reviews):# Tokenize text
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train_reviews)
    word_index = tokenizer.word_index
    vocab_size=len(word_index)
    # print(vocab_size)
    # Padding data
    sequences = tokenizer.texts_to_sequences(train_reviews)
    padded = pad_sequences(sequences, maxlen=29, padding='post', truncating='post')
    return padded, tokenizer

def convert_to_lower(text):
    # return the reviews after convering then to lowercase
    return text.str.lower()

def remove_punctuation(text):
    # return the reviews after removing punctuations
    text = text.str.replace(r'[^\w\s]', '')
    return text

def remove_stopwords(text):
    # return the reviews after removing the stopwords
    text = text.str.replace(pattern, '')
    return text

# def perform_tokenization(text):
#     # return the reviews after performing tokenization
#     text = text.apply(lambda row: word_tokenize(row))
#     return text

# def perform_padding(sequences):
#     # return the reviews after padding the reviews to maximum length
#     # sequences = tokenizer.texts_to_sequences(data)
#     # print(sequences[0])
#     # print(len(sequences[0]))
#     # print(sequences.shape)
#     padded =  pad_sequences(sequences, padding='post', truncating='post')
#     # print(padded)
#     # print(padded.shape)
#     return padded

def preprocess_data(data):
    # return processed data
    review = data["reviews"]
    review = convert_to_lower(review)
    review = remove_punctuation(review)
    review = remove_stopwords(review)
    # review = perform_tokenization(review)
    # review = encode_data(review)
    # review = perform_padding(review)
    # print(review.head(5))
    return review

# def softmax_activation(x):
#     # write your own implementation from scratch and return softmax values(using predefined softmax is prohibited)
#     exp_arr = tf.exp(x)
#     return exp_arr/tf.expand_dims(tf.reduce_sum(exp_arr, axis=1), -1)

def oversample_classes(df):
    max_size = df['ratings'].value_counts().max()
    lst = [df]
    for class_index, group in df.groupby('ratings'):
        lst.append(group.sample(max_size-len(group), replace=True).reset_index(drop=True))
    frame_new = pd.concat(lst)
    return frame_new.sample(frac=1)

def undersample_classes(df):
    #min_size = df['ratings'].value_counts().min()
    min_size = 6814
    lst = df.groupby('ratings').apply(lambda x: x.sample(min_size, replace=True)).reset_index(drop=True)
    return lst

class NeuralNet:

    def __init__(self, reviews, ratings, vocab_size, matrix):

        self.reviews = reviews
        self.ratings = ratings
        self.nn_model = Sequential()
        self.vocab_size = vocab_size
        self.embeddings_matrix = matrix

    def build_nn(self):
        #add the input and output layer here; you can use either tensorflow or pytorch
        self.nn_model.add(Embedding(self.vocab_size+1, 300, weights=[self.embeddings_matrix], trainable=False, input_length=29))
        # self.nn_model.add(tf.keras.layers.Flatten())
        # for neurons in [29, 27, 25, 23, 21, 19, 17, 15, 13, 11]:
        # for neurons in [29, 50, 100]:
        #     self.nn_model.add(Dense(neurons, activation='sigmoid', kernel_regularizer='l2'))
        
        # self.nn_model.add(tf.keras.layers.SimpleRNN(100, dropout=0.2))
        # self.nn_model.add(tf.keras.layers.LSTM(100,dropout=0.2))
        # self.nn_model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100, dropout=0.2)))
        # self.nn_model.add(tf.keras.layers.GRU(100, dropout=0.2))
        self.nn_model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(100, dropout=0.2)))
        # self.nn_model.add(Dense(5,activation='softmax'))
        self.nn_model.add(Dense(5, activation='softmax', kernel_regularizer='l2'))
        self.nn_model.compile(loss= tf.keras.losses.categorical_crossentropy, 
                              optimizer='adam',
                              metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

    def train_nn(self,batch_size,epochs):
        # write the training loop here; you can use either tensorflow or pytorch
        # print validation accuracy

        callbacks = []
        callbacks.append(EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True))
        callbacks.append(tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, min_lr=7.8125e-8, patience=25, min_delta=0.001, verbose=1))

        weight_df = pd.Series(np.argmax(self.ratings, axis=1))
        class_weight = 1/(weight_df.value_counts())*len(weight_df)/5
        class_weight = class_weight.to_dict()

        history = self.nn_model.fit(
            self.reviews, self.ratings, batch_size = batch_size,
            epochs = epochs, validation_split=0.2,
            callbacks=callbacks, verbose=1, 
            #class_weight=class_weight
            )
        res = self.nn_model.evaluate(self.reviews, self.ratings, verbose=1)
        print("Validation accuracy: {}%".format(round(res[1]*100, 2)))

    def predict(self, reviews, batch_size=1024):
        # return a list containing all the ratings predicted by the trained model
        results = self.nn_model.predict(reviews, batch_size=batch_size)
        return results


# DO NOT MODIFY MAIN FUNCTION'S PARAMETERS
def main(train_file):

    train_data = pd.read_csv(train_file, engine='python')
    
    #Uncomment following for oversampling
    # train_data = oversample_classes(train_data)
    # train_data = undersample_classes(train_data)
    train_data = train_data.sample(frac=1)
    train_ratings = train_data['ratings']
    train_reviews = preprocess_data(train_data)
    
    train_ratings = tf.keras.utils.to_categorical(train_ratings-1, num_classes=5)

    #padded_train, tokenizer = token(train_reviews)
    #word_index = tokenizer.word_index
    #vocab_size=len(word_index)
    padded_train, embeddings_matrix, vocab_size = embedding(train_reviews)
    return padded_train, embeddings_matrix, vocab_size, train_ratings

def train_new(padded_train, embeddings_matrix, vocab_size, train_ratings):
    batch_size,epochs=1024, 100
    model = NeuralNet(padded_train, train_ratings, vocab_size, embeddings_matrix)
    model.build_nn()
    try:
        model.train_nn(batch_size,epochs)
    except Exception as e:
        print(e)
    return model
  
def test_model(test_file, model):
    test_data = pd.read_csv(test_file, engine='python')
    test_reviews = preprocess_data(test_data)
    padded_test, tokenizer = token(test_reviews)
    test_pred = model.predict(padded_test)
    return np.argmax(test_pred, axis=1) + 1


In [ ]:
padded_train, embeddings_matrix, vocab_size, train_ratings = main("/content/drive/MyDrive/CS772/augmented.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
#Simple RNN with 100 and oversampling and batch size 1024k
model = train_new(padded_train, embeddings_matrix, vocab_size, train_ratings)

Epoch 1/100
27/27 [==============================] - 6s 88ms/step - loss: 1.6458 - accuracy: 0.2701 - recall: 0.0028 - precision: 0.4052 - val_loss: 1.4429 - val_accuracy: 0.3852 - val_recall: 0.1104 - val_precision: 0.5123
Epoch 2/100
27/27 [==============================] - 2s 66ms/step - loss: 1.4341 - accuracy: 0.3973 - recall: 0.0844 - precision: 0.5734 - val_loss: 1.3654 - val_accuracy: 0.4278 - val_recall: 0.2319 - val_precision: 0.5708
Epoch 3/100
27/27 [==============================] - 2s 66ms/step - loss: 1.3817 - accuracy: 0.4184 - recall: 0.1671 - precision: 0.5795 - val_loss: 1.3130 - val_accuracy: 0.4541 - val_recall: 0.2464 - val_precision: 0.5711
Epoch 4/100
27/27 [==============================] - 2s 66ms/step - loss: 1.3257 - accuracy: 0.4469 - recall: 0.2017 - precision: 0.5958 - val_loss: 1.2824 - val_accuracy: 0.4643 - val_recall: 0.2438 - val_precision: 0.5824
Epoch 5/100
27/27 [==============================] - 2s 67ms/step - loss: 1.3002 - accuracy: 0.4542 - re

In [ ]:
#Simple RNN with 100 and undersampling and batch size 1k

test_pred = test_model("/content/drive/MyDrive/CS772/gold_test.csv", model)
ypred = pd.Series(test_pred)
test_file = pd.read_csv('/content/drive/MyDrive/CS772/gold_test.csv', engine='python')
test_ratings = test_file["ratings"]
 
accuracy = metrics.accuracy_score(test_ratings, ypred)
print('Accuracy: %f' % accuracy)
 
precision = metrics.precision_score(test_ratings, ypred, average='weighted')
print('Precision: %f' % precision)
 
recall = metrics.recall_score(test_ratings, ypred, average='weighted')
print('Recall: %f' % recall)
 
f1 = metrics.f1_score(test_ratings, ypred, average='weighted')
print('F1 score: %f' % f1)

Accuracy: 0.183000
Precision: 0.383181
Recall: 0.183000
F1 score: 0.207493


In [ ]:
model.nn_model.save("/content/drive/MyDrive/CS772/rnn.h5")

In [ ]:
#LSTM with 100 batch size 1024k
model = train_new(padded_train, embeddings_matrix, vocab_size, train_ratings)

Epoch 1/100
27/27 [==============================] - 31s 87ms/step - loss: 1.6399 - accuracy: 0.2794 - recall_1: 0.0051 - precision_1: 0.2555 - val_loss: 1.4133 - val_accuracy: 0.4152 - val_recall_1: 0.0022 - val_precision_1: 0.6818
Epoch 2/100
27/27 [==============================] - 1s 54ms/step - loss: 1.3807 - accuracy: 0.4197 - recall_1: 0.1143 - precision_1: 0.6131 - val_loss: 1.3094 - val_accuracy: 0.4596 - val_recall_1: 0.1699 - val_precision_1: 0.6796
Epoch 3/100
27/27 [==============================] - 1s 55ms/step - loss: 1.3019 - accuracy: 0.4607 - recall_1: 0.1998 - precision_1: 0.6375 - val_loss: 1.2523 - val_accuracy: 0.4836 - val_recall_1: 0.1945 - val_precision_1: 0.6966
Epoch 4/100
27/27 [==============================] - 1s 54ms/step - loss: 1.2499 - accuracy: 0.4851 - recall_1: 0.2055 - precision_1: 0.6654 - val_loss: 1.2210 - val_accuracy: 0.4975 - val_recall_1: 0.2435 - val_precision_1: 0.6503
Epoch 5/100
27/27 [==============================] - 1s 55ms/step - los

In [ ]:
#LSTM 100 with undersampling

test_pred = test_model("/content/drive/MyDrive/CS772/gold_test.csv", model)
ypred = pd.Series(test_pred)
test_file = pd.read_csv('/content/drive/MyDrive/CS772/gold_test.csv', engine='python')
test_ratings = test_file["ratings"]
 
test_ratings = test_file["ratings"]
 
accuracy = metrics.accuracy_score(test_ratings, ypred)
print('Accuracy: %f' % accuracy)
 
precision = metrics.precision_score(test_ratings, ypred, average='weighted')
print('Precision: %f' % precision)
 
recall = metrics.recall_score(test_ratings, ypred, average='weighted')
print('Recall: %f' % recall)
 
f1 = metrics.f1_score(test_ratings, ypred, average='weighted')
print('F1 score: %f' % f1)

Accuracy: 0.271600
Precision: 0.373409
Recall: 0.271600
F1 score: 0.302217


In [ ]:
#Bi LSTM with 100 and oversampling and batch size 1024
model = train_new(padded_train, embeddings_matrix, vocab_size, train_ratings)

Epoch 1/100
27/27 [==============================] - 8s 139ms/step - loss: 1.6013 - accuracy: 0.3160 - recall_2: 0.0215 - precision_2: 0.4367 - val_loss: 1.3134 - val_accuracy: 0.4635 - val_recall_2: 0.1741 - val_precision_2: 0.6693
Epoch 2/100
27/27 [==============================] - 3s 94ms/step - loss: 1.3016 - accuracy: 0.4772 - recall_2: 0.1857 - precision_2: 0.6712 - val_loss: 1.2428 - val_accuracy: 0.4927 - val_recall_2: 0.2443 - val_precision_2: 0.6547
Epoch 3/100
27/27 [==============================] - 3s 94ms/step - loss: 1.2201 - accuracy: 0.5097 - recall_2: 0.2529 - precision_2: 0.6678 - val_loss: 1.1947 - val_accuracy: 0.5095 - val_recall_2: 0.2785 - val_precision_2: 0.6599
Epoch 4/100
27/27 [==============================] - 3s 94ms/step - loss: 1.1733 - accuracy: 0.5251 - recall_2: 0.2887 - precision_2: 0.6743 - val_loss: 1.1572 - val_accuracy: 0.5249 - val_recall_2: 0.3224 - val_precision_2: 0.6552
Epoch 5/100
27/27 [==============================] - 3s 94ms/step - los

In [ ]:
#Bi LSTM 100 with undersampling 
test_pred = test_model("/content/drive/MyDrive/CS772/gold_test.csv", model)
ypred = pd.Series(test_pred)
test_file = pd.read_csv('/content/drive/MyDrive/CS772/gold_test.csv', engine='python')
test_ratings = test_file["ratings"]
 
test_ratings = test_file["ratings"]
 
accuracy = metrics.accuracy_score(test_ratings, ypred)
print('Accuracy: %f' % accuracy)
 
precision = metrics.precision_score(test_ratings, ypred, average='weighted')
print('Precision: %f' % precision)
 
recall = metrics.recall_score(test_ratings, ypred, average='weighted')
print('Recall: %f' % recall)
 
f1 = metrics.f1_score(test_ratings, ypred, average='weighted')
print('F1 score: %f' % f1)

Accuracy: 0.269800
Precision: 0.397168
Recall: 0.269800
F1 score: 0.305369


In [ ]:
#GRU with 100 and oversampling and batch size 1024
model = train_new(padded_train, embeddings_matrix, vocab_size, train_ratings)

Epoch 1/100
27/27 [==============================] - 4s 79ms/step - loss: 1.6945 - accuracy: 0.2000 - recall_3: 0.0000e+00 - precision_3: 0.0000e+00 - val_loss: 1.6723 - val_accuracy: 0.1967 - val_recall_3: 0.0000e+00 - val_precision_3: 0.0000e+00
Epoch 2/100
27/27 [==============================] - 1s 48ms/step - loss: 1.6165 - accuracy: 0.2426 - recall_3: 0.0241 - precision_3: 0.3546 - val_loss: 1.4110 - val_accuracy: 0.3610 - val_recall_3: 0.0065 - val_precision_3: 0.6984
Epoch 3/100
27/27 [==============================] - 1s 48ms/step - loss: 1.3914 - accuracy: 0.3648 - recall_3: 0.0344 - precision_3: 0.6726 - val_loss: 1.3376 - val_accuracy: 0.3895 - val_recall_3: 0.0894 - val_precision_3: 0.6259
Epoch 4/100
27/27 [==============================] - 1s 48ms/step - loss: 1.3281 - accuracy: 0.3927 - recall_3: 0.1360 - precision_3: 0.6247 - val_loss: 1.2786 - val_accuracy: 0.4338 - val_recall_3: 0.1688 - val_precision_3: 0.6583
Epoch 5/100
27/27 [==============================] - 1s 

In [ ]:
#GRU 100 with undersampling
test_pred = test_model("/content/drive/MyDrive/CS772/test.csv", model)
ypred = pd.Series(test_pred)
test_file = pd.read_csv('/content/drive/MyDrive/CS772/gold_test.csv', engine='python')
test_ratings = test_file["ratings"]
 

 
accuracy = metrics.accuracy_score(test_ratings, ypred)
print('Accuracy: %f' % accuracy)
 
precision = metrics.precision_score(test_ratings, ypred, average='weighted')
print('Precision: %f' % precision)
 
recall = metrics.recall_score(test_ratings, ypred, average='weighted')
print('Recall: %f' % recall)
 
f1 = metrics.f1_score(test_ratings, ypred, average='weighted')
print('F1 score: %f' % f1)

Accuracy: 0.217300
Precision: 0.394734
Recall: 0.217300
F1 score: 0.254253


In [ ]:
#bi GRU with 100 and batch size 1024
model = train_new(padded_train, embeddings_matrix, vocab_size, train_ratings)

Epoch 1/100
8/8 [==============================] - 6s 252ms/step - loss: 1.5488 - accuracy: 0.4274 - recall_4: 0.0620 - precision_4: 0.5061 - val_loss: 1.2850 - val_accuracy: 0.5765 - val_recall_4: 0.5765 - val_precision_4: 0.5835
Epoch 2/100
8/8 [==============================] - 1s 88ms/step - loss: 1.2720 - accuracy: 0.5771 - recall_4: 0.5734 - precision_4: 0.5908 - val_loss: 1.1991 - val_accuracy: 0.5870 - val_recall_4: 0.5290 - val_precision_4: 0.6857
Epoch 3/100
8/8 [==============================] - 1s 89ms/step - loss: 1.1829 - accuracy: 0.5990 - recall_4: 0.5002 - precision_4: 0.7301 - val_loss: 1.1386 - val_accuracy: 0.6155 - val_recall_4: 0.4730 - val_precision_4: 0.7556
Epoch 4/100
8/8 [==============================] - 1s 89ms/step - loss: 1.1089 - accuracy: 0.6213 - recall_4: 0.4971 - precision_4: 0.7675 - val_loss: 1.0729 - val_accuracy: 0.6265 - val_recall_4: 0.5340 - val_precision_4: 0.7386
Epoch 5/100
8/8 [==============================] - 1s 88ms/step - loss: 1.0517 

In [ ]:
#Bi GRU with undersampling and batch 2048
test_pred = test_model("/content/drive/MyDrive/CS772/gold_test.csv", model)
test_file = pd.read_csv('/content/drive/MyDrive/CS772/gold_test.csv', engine='python')
test_ratings = test_file["ratings"]
ypred = pd.Series(test_pred)

 
test_ratings = test_file["ratings"]
 
accuracy = metrics.accuracy_score(test_ratings, ypred)
print('Accuracy: %f' % accuracy)
 
precision = metrics.precision_score(test_ratings, ypred, average='weighted')
print('Precision: %f' % precision)
 
recall = metrics.recall_score(test_ratings, ypred, average='weighted')
print('Recall: %f' % recall)
 
f1 = metrics.f1_score(test_ratings, ypred, average='weighted')
print('F1 score: %f' % f1)

InvalidArgumentError: ignored

In [ ]:
test_pred

array([3, 2, 5, ..., 5, 5, 5])

Assignment 2 cells


In [ ]:
model, test_pred = main("/content/drive/MyDrive/CS772/train.csv", "/content/drive/MyDrive/CS772/test.csv")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Epoch 1/500
4/4 [==============================] - 1s 187ms/step - loss: 2.4322 - accuracy: 0.1530 - recall_4: 5.0000e-05 - precision_4: 0.4000 - val_loss: 1.9113 - val_accuracy: 0.5321 - val_recall_4: 0.0406 - val_precision_4: 0.6893
Epoch 2/500
4/4 [==============================] - 0s 88ms/step - loss: 1.6847 - accuracy: 0.6670 - recall_4: 0.2387 - precision_4: 0.7192 - val_loss: 1.7208 - val_accuracy: 0.5733 - val_recall_4: 0.4343 - val_precision_4: 0.5974
Epoch 3/500
4/4 [==============================] - 0s 88ms/step - loss: 1.4668 - accuracy: 0.6873 - recall_4: 0.5991 - precision_4: 0.6970 - val_loss: 1.6609 - val_accuracy: 0.5740 - val_recall_4: 0.5345 - val_precision_4: 0.5821
Epoch 4/500
4/4 [==============================] - 0s 87ms/step - loss: 1.3878 - accuracy: 0.6861 - recall_4: 0.6634 - precision_4: 0.6905 - val_loss: 1.6183 - val_accuracy: 0.5742 - val_recall_4: 0.5564 - val_precision_4: 0.5781
Epoch 5/500
4/4 [==============================] - 0s 86ms/step - loss: 1.3

In [ ]:
model.nn_model.save("/content/drive/MyDrive/CS772/w2vec_1layer_sigmoid.h5")

In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/CS772/gold_test.csv', engine='python')
test_ratings = test_data["ratings"]
test_reviews = preprocess_data(test_data)
test_ratings = tf.keras.utils.to_categorical(test_ratings-1, num_classes=5)
padded_test, blah1, blah2 = embedding(test_reviews)
model.nn_model.evaluate(padded_test, test_ratings)

 33/313 [==>...........................] - ETA: 0s - loss: 1.4603 - accuracy: 0.5568 - recall_4: 0.4981 - precision_4: 0.5560

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


313/313 [==============================] - 1s 3ms/step - loss: 1.4015 - accuracy: 0.5785 - recall_4: 0.5251 - precision_4: 0.5838


[1.4015237092971802, 0.578499972820282, 0.5250999927520752, 0.5838336944580078]

In [ ]:
ypred = pd.Series(test_pred)
test_file = pd.read_csv('/content/drive/MyDrive/CS772/gold_test.csv', engine='python')
 
test_ratings = test_file["ratings"]
 
accuracy = metrics.accuracy_score(test_ratings, ypred)
print('Accuracy: %f' % accuracy)
 
precision = metrics.precision_score(test_ratings, ypred, average='weighted')
print('Precision: %f' % precision)
 
recall = metrics.recall_score(test_ratings, ypred, average='weighted')
print('Recall: %f' % recall)
 
f1 = metrics.f1_score(test_ratings, ypred, average='weighted')
print('F1 score: %f' % f1)

Accuracy: 0.578300
Precision: 0.366492
Recall: 0.578300
F1 score: 0.424529


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
